In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose

Agrupando os ações por segmento de atuação

In [2]:
df_hist_cot_norm = pd.read_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_hist_cot_norm.csv")
df_hist_cot_norm["Date"] = pd.to_datetime(df_hist_cot_norm["Date"])
df_hist_cot_norm.set_index("Date", inplace=True)

In [3]:
# Listando os segmentos de mercado
df_segmentos = pd.read_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\info_empresas.csv")[["tickers","segmento"]]
df_segmentos.set_index("tickers", inplace=True)
lista_segmentos = list(set(df_segmentos["segmento"]))

In [4]:
for segmento in lista_segmentos:
    
    df_segmento = pd.DataFrame(index=df_hist_cot_norm.index)
    
    for ticker in df_hist_cot_norm.columns:
    
        if df_segmentos.loc[ticker]["segmento"] == segmento:
            
            df_segmento[ticker] = df_hist_cot_norm[ticker]
    
    df_segmento.to_csv(f"C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\hist_cot_norm_segmentos\\{segmento}.csv")

Gerando um df com o histórico de variação média de cada segmento

In [5]:
data_inicial = '2013-01-01'
data_final = '2023-10-01'
datas = pd.date_range(start=data_inicial, end=data_final, freq='D')
df_media_segmentos = pd.DataFrame({'Date': datas}).set_index('Date')

for segmento in lista_segmentos:

    df_hist_cot_ticker = pd.read_csv(
        f"C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\hist_cot_norm_segmentos\\{segmento}.csv", 
        index_col="Date")
    
    for data in df_hist_cot_ticker.index:
        
        soma_linha = 0
        cont_tickers_somados = 0
        
        for ticker in df_hist_cot_ticker.columns:
            
            cond_calc_media = df_hist_cot_ticker.loc[data, ticker] !=0 and not np.isnan(df_hist_cot_ticker.loc[data, ticker])
            if cond_calc_media:
                
                soma_linha += df_hist_cot_ticker.loc[data, ticker]
                cont_tickers_somados += 1
        
        if cont_tickers_somados > 0:
            df_hist_cot_ticker.loc[data, "media"] = soma_linha / cont_tickers_somados 
    
    for data in df_hist_cot_ticker.index:
        df_media_segmentos.loc[data, segmento] = df_hist_cot_ticker.loc[data, "media"]

Trantando valores faltantes

In [6]:
# Removendo datas sem cotação (finais de semana, feriados, ...)
df_media_segmentos.dropna(inplace=True, how="all")

# Tratando datas anteriores à data de referencia (data inicial de amostragem)
for segmento in df_media_segmentos.columns:

    data_ref = df_media_segmentos[segmento].first_valid_index()

    for data in df_media_segmentos.index:

        if data < data_ref:

            df_media_segmentos.loc[data, segmento] = 0

# Tratando datas com cotações faltantes
for segmento in df_media_segmentos.columns:
    df_media_segmentos[f"{segmento}"].ffill(inplace=True)

In [7]:
df_media_segmentos

,Material Aeronáutico e de Defesa,Produtos para Construção,Papel e Celulose,Bancos,Serviços Diversos,Embalagens,Acessórios,Transporte Rodoviário,Químicos Diversos,Programas de Fidelização,...,Cervejas e Refrigerantes,Artefatos de Cobre,Alimentos,Holdings Diversificadas,Medicamentos e Outros Produtos,Brinquedos e Jogos,Petroquímicos,Programas e Serviços,Material de Transporte,Produtos de Limpeza
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2013-01-03,0.988459,1.010012,1.004065,1.014291,1.006439,1.032258,0.998255,1.008572,1.000000,0.000000,...,1.017730,0.936759,1.000000,0.993357,0.994568,1.020833,1.000000,0.986203,1.056180,1.000000
2013-01-04,0.968092,1.002314,1.004065,1.015278,1.001439,1.051613,0.954403,1.018286,1.038462,0.000000,...,1.000952,0.891304,1.000000,0.996503,0.996902,1.020833,1.000000,0.963746,1.056180,0.943396
2013-01-07,0.976239,1.002908,1.002710,1.017598,1.001843,1.000000,0.974215,1.017143,1.038462,0.000000,...,1.012494,0.879447,1.000000,1.154772,0.995100,1.020833,1.000000,0.958496,1.056180,0.991195
2013-01-08,0.934827,1.009705,1.002710,1.027687,0.989471,1.000000,0.948975,0.999714,1.030769,0.000000,...,1.011304,0.905138,1.000000,1.154772,0.993418,1.000000,1.000000,0.969360,1.056180,0.944654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-25,1.239874,1.322343,2.721097,4.275205,0.621748,12.566089,1.401785,1.040917,31.078582,0.084091,...,1.159669,0.060608,0.575768,3.445548,1.521567,6.041667,1.915820,1.386308,2.980277,0.226415
2023-09-26,1.241324,1.303498,2.690933,4.196466,0.616584,12.239569,1.395628,1.025984,30.463575,0.086364,...,1.143879,0.059997,0.565671,3.425276,1.500672,6.041667,1.915820,1.375510,2.978338,0.221384
2023-09-27,1.228998,1.296135,2.696153,4.169864,0.614000,12.209885,1.404662,1.017272,30.756434,0.084091,...,1.137739,0.060119,0.572839,3.417723,1.500804,6.041667,1.905905,1.378763,2.973075,0.221384


Exportando os histórios das médias das cotações normalizadas de cada segmento

In [8]:
df_media_segmentos.to_csv(f"C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_media_segmentos.csv", index=True)

Decompondo as séries temporais das médias das cotações normalizadas de cada segmento

In [9]:
df_tend_media_segmentos = pd.DataFrame(index = df_media_segmentos.index)
df_saz_media_segmentos = pd.DataFrame(index = df_media_segmentos.index)
df_resid_media_segmentos = pd.DataFrame(index = df_media_segmentos.index)

for segmento in df_media_segmentos.columns:

    # 22 dias de cotação por mês / 252 dias de cotação por ano / df_media_segm.shape[0]//2 maior período possível para decomposição da base de dados
    
    df_tend_media_segmentos[f"{segmento}"] = seasonal_decompose(df_media_segmentos[f"{segmento}"],
                                                period = 252,
                                                model = "additive").trend
    df_saz_media_segmentos[f"{segmento}"] = seasonal_decompose(df_media_segmentos[f"{segmento}"],
                                                period = 252,
                                                model = "additive").seasonal
    df_resid_media_segmentos[f"{segmento}"] = seasonal_decompose(df_media_segmentos[f"{segmento}"],
                                                period = 252,
                                                model = "additive").resid

df_tend_media_segmentos.dropna(inplace=True)
df_saz_media_segmentos.dropna(inplace=True)
df_resid_media_segmentos.dropna(inplace=True)

In [10]:
df_tend_media_segmentos

,Material Aeronáutico e de Defesa,Produtos para Construção,Papel e Celulose,Bancos,Serviços Diversos,Embalagens,Acessórios,Transporte Rodoviário,Químicos Diversos,Programas de Fidelização,...,Cervejas e Refrigerantes,Artefatos de Cobre,Alimentos,Holdings Diversificadas,Medicamentos e Outros Produtos,Brinquedos e Jogos,Petroquímicos,Programas e Serviços,Material de Transporte,Produtos de Limpeza
Date,,,,,,,,,,,,,,,,,,,,,
2013-06-28,1.232411,1.064337,0.998356,1.067058,0.937099,1.504447,0.737670,0.757953,1.053574,0.000000,...,1.008987,0.971050,1.007571,1.111866,1.173793,0.774678,1.002908,0.951048,0.983414,0.875936
2013-07-01,1.233544,1.064432,0.998289,1.067306,0.936575,1.508830,0.736059,0.756129,1.053793,0.000000,...,1.009075,0.971167,1.007621,1.112400,1.174585,0.772900,1.002943,0.950737,0.983035,0.875057
2013-07-02,1.234770,1.064525,0.998215,1.067523,0.936052,1.513181,0.734522,0.754264,1.053984,0.000000,...,1.009147,0.971501,1.007672,1.112966,1.175372,0.771123,1.002979,0.950479,0.982544,0.874291
2013-07-03,1.236053,1.064662,0.998143,1.067739,0.935543,1.517666,0.733019,0.752357,1.054133,0.000000,...,1.009232,0.972046,1.007723,1.113298,1.176150,0.769345,1.003015,0.950262,0.982054,0.873617
2013-07-04,1.237414,1.064839,0.998074,1.067934,0.935144,1.522604,0.731561,0.750466,1.054329,0.000000,...,1.009342,0.972626,1.007773,1.113451,1.176954,0.767526,1.003051,0.950078,0.981563,0.872988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-24,1.220680,1.643790,2.457730,3.781415,0.545387,8.409167,1.215195,0.858695,31.955046,0.094154,...,1.263077,0.061078,0.783330,3.501298,1.545657,5.246127,1.955120,1.191225,2.256687,0.182140
2023-03-27,1.221814,1.640610,2.459365,3.783334,0.545581,8.430012,1.217054,0.859969,31.947833,0.093843,...,1.262515,0.061013,0.781666,3.499681,1.545343,5.253609,1.956094,1.191706,2.260279,0.182333
2023-03-28,1.223100,1.637620,2.461037,3.785297,0.545801,8.450328,1.218964,0.861225,31.940406,0.093546,...,1.261977,0.060953,0.780089,3.498132,1.545029,5.261091,1.957056,1.192281,2.263856,0.182527


In [11]:
df_tend_media_segmentos.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_tend_media_segmentos.csv")
df_saz_media_segmentos.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_saz_media_segmentos.csv")
df_resid_media_segmentos.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_resid_media_segmentos.csv")

Gerando a matriz de correlação entre os segmentos

In [12]:
mtx_corr_segmentos = df_media_segmentos.corr()
mtx_corr_segmentos.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\mtx_corr_segmentos.csv")

Gerando a matriz de correlação entre as sazonalidades da médias dos segmentos

In [13]:
mtx_corr_saz_segmentos = df_saz_media_segmentos.corr()
mtx_corr_saz_segmentos.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\mtx_corr_saz_segmentos.csv")